In [1]:
# Homework 1
#import keras
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix   
import numpy as np

In [2]:
# Load data and data standardization
def load_data():
    '''Load the MNIST dataset'''
    
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    return (X_train, y_train, X_test, y_test)

def data_std(X_train, X_test):
    '''Data standardization
    
    Parameters
    ----------
    X_train: origianl training set
    X_test: original test set
    
    Returns
    -------
    X_train_std: rescaled training set
    X_test_std: rescaled test set
    '''
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    
    return X_train_std, X_test_std

In [3]:
def create_NN(n_features = 784, n_outputs = 10): # 30 points
    '''create a deep feedforwad neural network using keras
    
    Parameters
    -----------
    n_features: the number of input features/units
    n_output: the number of output units
    
    Returns
    -------
    myNN: the neural network model
    
    '''
    ## add your code here
    myNN = tf.keras.models.Sequential()

    myNN.add(tf.keras.layers.Dense(
        units = 50,
        input_dim = n_features,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        activation = 'sigmoid'))
    
    myNN.add(tf.keras.layers.Dense(
        units = 10,
        input_dim = 50,
        kernel_initializer = 'glorot_uniform',
        bias_initializer = 'zeros',
        activation = 'softmax'))
    
    sgd_optimizer = tf.keras.optimizers.SGD(lr=0.001, decay=1e-7, momentum=.9)
    myNN.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy',metrics=["accuracy"])
    ##
    
    return myNN

In [4]:
def nn_params_search(nn, X_train, y_train): # 30 points
    '''Search best paramaters
    
    Parameters
    ----------
    X_train: features
    y_train: target of the input

    
    Returns
    -------
    best_params_
    
    Example grid: (you can customize the search graid by youself)
    param_grid = [{'batch_size': [64, 128], 'epochs' : [10, 30, 50]}]
        
    '''
    ## add your code here
    param_grid = {
        'epochs' : [50, 70, 100],
        'batch_size' : [32, 64, 128]
    }
    
    grid = GridSearchCV(
    estimator = nn,
    param_grid = param_grid,
    cv = 3,
    n_jobs=-1,
    #scoring = 'neg_log_loss')
    )
    
    gs_cv = grid.fit(X_train, y_train)
    
    gs_cv.best_params_
    
    ##
    
    return gs_cv.best_params_

In [5]:
def retrain_best_nn(best_params, X_train, y_train): # 10 points
    '''
    Retrain classifier using the best parameters
    
    Paramters
    ----------
    best_params:
    X_train: data input of the training set
    y_train: target of the input
    
    Returns
    ---------
    bestNN: the nn classifier trained using the best parameters
    
    '''
    ## add your code here  
    bestNN = create_NN()
    
    history = bestNN.fit(X_train, y_train,
                    batch_size=best_params['batch_size'], epochs=best_params['epochs'],
                    verbose=1,
                    validation_split=0.1)
    ##
    
    return bestNN

In [6]:
def performance_acc(y, y_pred): # 10 points
    ''' calculate the concusion matrix and average accuracy
    
        Parameters
        ----------
        y: real target
        y_pred: prediction
        
        Returns
        -------
        cm: confusion matrix
        acc: accuracy
    '''
    ## add your code here
    cm = confusion_matrix(y, y_pred)
    
    correct_preds = np.sum(y == y_pred, axis = 0)
    acc = correct_preds / y_pred.shape[0]
    ##
    
    return cm, acc

In [7]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

if __name__ == '__main__':
    
    #Task 1. load the dataset
    (X_train, y_train, X_test, y_test) = load_data()
    
    # 1.1 reshape the training and test sets to N * 784. 10 points
    ## add your code here
    X_train_1 = np.reshape(X_train, [X_train.shape[0], X_train.shape[1] * X_train.shape[2]])
    X_test_1 = np.reshape(X_test, [X_test.shape[0], X_test.shape[1] * X_test.shape[2]])
    print('1. X_train: {}, X_train_1: {}'.format(X_train.shape, X_train_1.shape))
    ##
    
    # 1.2 transform y_train to one-hot vectors using keras.utils.to_categorical. 10 points
    ## add your code here
    y_train_onehot = tf.keras.utils.to_categorical(y_train)
    print('y_train_onehot: {}'.format(y_train_onehot.shape))
    ##
    
    #Task 2. create a deep feedforward neural network
    myNN = create_NN(X_train_1.shape[1], y_train_onehot.shape[1])
    myNN.summary()
    myNN1 = KerasClassifier(build_fn = create_NN, batch_size = 64, epochs = 50)
    
    #Task 3. Search best paprameters, and report the performance
    best_params = nn_params_search(myNN1, X_train_1, y_train)
    print('Best parameters: ', best_params)
    
    bestNN = retrain_best_nn(best_params, X_train_1, y_train_onehot)
    y_test_pred = bestNN.predict_classes(X_test_1)
    cm, acc = performance_acc(y_test, y_test_pred)
    print('Confusion matrix:\n', cm)
    print('Accuracy =    {:.3f}%'.format(acc*100))
    
    #Task 4. Search best nn parameters after data standardization, and report the performance
    X_train_std, X_test_std = data_std(X_train_1, X_test_1)
    
    best_params =  nn_params_search(myNN1, X_train_std, y_train)
    print('Best parameters: ', best_params)
    
    bestNN_std = retrain_best_nn(best_params, X_train_std, y_train_onehot)
    y_test_std_pred = bestNN_std.predict_classes(X_test_std)
    cm1, acc1 = performance_acc(y_test, y_test_std_pred)
    print('Confusion matrix:\n', cm1)
    print('Accuracy =    {:.3f}%'.format(acc1*100))

1. X_train: (60000, 28, 28), X_train_1: (60000, 784)
y_train_onehot: (60000, 10)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples
Epoch 1/100
60000/60000 [==============================] - 3s 53us/sample - loss: 1.0104 - accuracy: 0.7486
Epoch 2/100
60000/60000 [==============================] - 2s 40us/sample - loss: 0.5242 - accuracy: 0.8777
Epoch 3/100
60000/60000 [==============================] - 2s 37us/sample - loss: 0.4258 - accuracy: 0.8931
Epoch 4/100
60000/60000 [==============================] - 2s 36us/sample - loss: 0.37

60000/60000 [==============================] - 2s 36us/sample - loss: 0.1505 - accuracy: 0.9568
Epoch 70/100
60000/60000 [==============================] - 2s 34us/sample - loss: 0.1501 - accuracy: 0.9570s - loss: 0.1504 - accuracy
Epoch 71/100
60000/60000 [==============================] - 2s 34us/sample - loss: 0.1498 - accuracy: 0.9569
Epoch 72/100
60000/60000 [==============================] - 2s 34us/sample - loss: 0.1484 - accuracy: 0.9562
Epoch 73/100
60000/60000 [==============================] - 2s 34us/sample - loss: 0.1470 - accuracy: 0.9564
Epoch 74/100
60000/60000 [==============================] - 2s 37us/sample - loss: 0.1488 - accuracy: 0.9568
Epoch 75/100
60000/60000 [==============================] - 3s 43us/sample - loss: 0.1481 - accuracy: 0.9556
Epoch 76/100
60000/60000 [==============================] - 2s 36us/sample - loss: 0.1477 - accuracy: 0.9568
Epoch 77/100
60000/60000 [==============================] - 2s 36us/sample - loss: 0.1448 - accuracy: 0.9577
Epoch

54000/54000 [==============================] - 2s 39us/sample - loss: 0.2147 - accuracy: 0.9369 - val_loss: 0.1960 - val_accuracy: 0.9443
Epoch 32/100
54000/54000 [==============================] - 2s 38us/sample - loss: 0.2085 - accuracy: 0.9385 - val_loss: 0.1883 - val_accuracy: 0.9440
Epoch 33/100
54000/54000 [==============================] - 2s 38us/sample - loss: 0.2114 - accuracy: 0.9389 - val_loss: 0.1948 - val_accuracy: 0.9465
Epoch 34/100
54000/54000 [==============================] - 2s 37us/sample - loss: 0.2112 - accuracy: 0.9389 - val_loss: 0.1771 - val_accuracy: 0.9502
Epoch 35/100
54000/54000 [==============================] - 2s 36us/sample - loss: 0.2035 - accuracy: 0.9407 - val_loss: 0.1831 - val_accuracy: 0.9453
Epoch 36/100
54000/54000 [==============================] - 2s 36us/sample - loss: 0.2020 - accuracy: 0.9407 - val_loss: 0.1887 - val_accuracy: 0.9453
Epoch 37/100
54000/54000 [==============================] - 2s 36us/sample - loss: 0.2024 - accuracy: 0.940

54000/54000 [==============================] - 2s 37us/sample - loss: 0.1562 - accuracy: 0.9532 - val_loss: 0.1471 - val_accuracy: 0.9570
Epoch 86/100
54000/54000 [==============================] - 2s 36us/sample - loss: 0.1596 - accuracy: 0.9520 - val_loss: 0.1521 - val_accuracy: 0.9568
Epoch 87/100
54000/54000 [==============================] - 2s 43us/sample - loss: 0.1529 - accuracy: 0.9538 - val_loss: 0.1514 - val_accuracy: 0.9560
Epoch 88/100
54000/54000 [==============================] - 2s 41us/sample - loss: 0.1463 - accuracy: 0.9564 - val_loss: 0.1501 - val_accuracy: 0.9553
Epoch 89/100
54000/54000 [==============================] - 2s 41us/sample - loss: 0.1472 - accuracy: 0.9560 - val_loss: 0.1496 - val_accuracy: 0.9563
Epoch 90/100
54000/54000 [==============================] - 2s 38us/sample - loss: 0.1516 - accuracy: 0.9540 - val_loss: 0.1512 - val_accuracy: 0.9563
Epoch 91/100
54000/54000 [==============================] - 2s 38us/sample - loss: 0.1498 - accuracy: 0.954

/Users/blakelyfrechette/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Train on 60000 samples
Epoch 1/100
60000/60000 [==============================] - 6s 95us/sample - loss: 0.9864 - accuracy: 0.7708
Epoch 2/100
60000/60000 [==============================] - 4s 62us/sample - loss: 0.4664 - accuracy: 0.8850s - loss: 0.4699 - accuracy
Epoch 3/100
60000/60000 [==============================] - 246s 4ms/sample - loss: 0.3705 - accuracy: 0.9022
Epoch 4/100
60000/60000 [==============================] - 8s 139us/sample - loss: 0.3268 - accuracy: 0.9111
Epoch 5/100
60000/60000 [==============================] - 5s 86us/sample - loss: 0.3000 - accuracy: 0.9170
Epoch 6/100
60000/60000 [==============================] - 7s 108us/sample - loss: 0.2809 - accuracy: 0.9220
Epoch 7/100
60000/60000 [==============================] - 6s 98us/sample - loss: 0.2662 - accuracy: 0.9256
Epoch 8/100
60000/60000 [==============================] - 6s 106us/sample - loss: 0.2543 - accuracy: 0.9284
Epoch 9/100
60000/60000 [==============================] - 6s 94us/sample - loss: 

60000/60000 [==============================] - 3s 58us/sample - loss: 0.0908 - accuracy: 0.9763
Epoch 76/100
60000/60000 [==============================] - 4s 59us/sample - loss: 0.0900 - accuracy: 0.9768
Epoch 77/100
60000/60000 [==============================] - 4s 75us/sample - loss: 0.0892 - accuracy: 0.9772
Epoch 78/100
60000/60000 [==============================] - 5s 78us/sample - loss: 0.0884 - accuracy: 0.9772
Epoch 79/100
60000/60000 [==============================] - 4s 63us/sample - loss: 0.0876 - accuracy: 0.9776
Epoch 80/100
60000/60000 [==============================] - 4s 62us/sample - loss: 0.0868 - accuracy: 0.9779
Epoch 81/100
60000/60000 [==============================] - 4s 61us/sample - loss: 0.0861 - accuracy: 0.9783
Epoch 82/100
60000/60000 [==============================] - 4s 63us/sample - loss: 0.0853 - accuracy: 0.9783s - los
Epoch 83/100
60000/60000 [==============================] - 4s 62us/sample - loss: 0.0846 - accuracy: 0.9784
Epoch 84/100
60000/60000 

54000/54000 [==============================] - 4s 71us/sample - loss: 0.1450 - accuracy: 0.9596 - val_loss: 0.1486 - val_accuracy: 0.9592
Epoch 36/100
54000/54000 [==============================] - 4s 78us/sample - loss: 0.1430 - accuracy: 0.9603 - val_loss: 0.1477 - val_accuracy: 0.9595
Epoch 37/100
54000/54000 [==============================] - 5s 84us/sample - loss: 0.1411 - accuracy: 0.9608 - val_loss: 0.1467 - val_accuracy: 0.9597
Epoch 38/100
54000/54000 [==============================] - 4s 75us/sample - loss: 0.1392 - accuracy: 0.9613 - val_loss: 0.1458 - val_accuracy: 0.9605
Epoch 39/100
54000/54000 [==============================] - 5s 86us/sample - loss: 0.1373 - accuracy: 0.9620 - val_loss: 0.1450 - val_accuracy: 0.9605
Epoch 40/100
54000/54000 [==============================] - 5s 84us/sample - loss: 0.1356 - accuracy: 0.9626 - val_loss: 0.1439 - val_accuracy: 0.9602
Epoch 41/100
54000/54000 [==============================] - 4s 74us/sample - loss: 0.1338 - accuracy: 0.962

54000/54000 [==============================] - 5s 97us/sample - loss: 0.0832 - accuracy: 0.9789 - val_loss: 0.1251 - val_accuracy: 0.9645
Epoch 90/100
54000/54000 [==============================] - 4s 73us/sample - loss: 0.0826 - accuracy: 0.9793 - val_loss: 0.1249 - val_accuracy: 0.9655
Epoch 91/100
54000/54000 [==============================] - 4s 78us/sample - loss: 0.0819 - accuracy: 0.9792 - val_loss: 0.1250 - val_accuracy: 0.9648
Epoch 92/100
54000/54000 [==============================] - 6s 103us/sample - loss: 0.0812 - accuracy: 0.9794 - val_loss: 0.1249 - val_accuracy: 0.9648
Epoch 93/100
54000/54000 [==============================] - 5s 84us/sample - loss: 0.0806 - accuracy: 0.9798 - val_loss: 0.1249 - val_accuracy: 0.9653
Epoch 94/100
54000/54000 [==============================] - 4s 75us/sample - loss: 0.0800 - accuracy: 0.9801 - val_loss: 0.1246 - val_accuracy: 0.9653
Epoch 95/100
54000/54000 [==============================] - 6s 105us/sample - loss: 0.0793 - accuracy: 0.9